In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteria, StoppingCriteriaList
from bamboo import bamboo

class StopTokenCriteria(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        # 检查最新生成的 token 是否是停止标记
        return input_ids[0, -1] in self.stop_token_id

# Load the tokenizer and model
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#tokenizer = AutoTokenizer.from_pretrained("/home/dozhang/nlcmt/HuggingfaceModels/Llama-2-13b-chat-hf")
#model = AutoModelForCausalLM.from_pretrained("/home/dozhang/nlcmt/HuggingfaceModels/Llama-2-13b-chat-hf", torch_dtype=torch.bfloat16, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained("/home/dozhang/nlcmt1/HuggingfaceModels/Meta-Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("/home/dozhang/nlcmt1/HuggingfaceModels/Meta-Llama-3.1-8B-Instruct", torch_dtype=torch.float16, device_map="auto")

#tokenizer = AutoTokenizer.from_pretrained("/home/dozhang/nlcmt1/HuggingfaceModels/Meta-Llama-3.1-70B-Instruct")
#model = AutoModelForCausalLM.from_pretrained("/home/dozhang/nlcmt1/HuggingfaceModels/Meta-Llama-3.1-70B-Instruct", torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:


#input_text = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n2+5=10, 3+6=18, 4+7=?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

#input_text = "Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?"


#input_text = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nif let 2+3=13, 3+4=25, 4+5=41, 5+6=?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

#input_text = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#input_text = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhich of these is needed in all stages of the butterfly's life cycle?\nA. Wings\nB. Eyes\nC. Soil\nD. Air<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#input_text="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhich statement about the molecules in ice and the molecules in liquid water is correct?\nA. The molecules in ice have more energy than the molecules in liquid water.\nB. The molecules in ice contain different atoms than the molecules in liquid water.\nC. The molecules in ice have more electric charge than the molecules in liquid water.\nD. The molecules in ice are less free to move than the molecules in liquid water.<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#input_text="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
input_text="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a math problem solver focused on word problems involving rates, time, and productivity. Calculate the missing values based on the details provided, ensuring accuracy by accounting for all workers' productivity and shifts. Break down each worker’s contribution over the specified time, and determine how many toys the fifth worker produces per hour to meet the remaining order requirements.<|eot_id|><|start_header_id|>user<|end_header_id|>A toy manufacturer receives an order for 400 toys. 5 workers are available to work on the order. 2 of the workers produce 6 toys an hour, and another 2 workers produce 4 toys an hour. They all work on the order during their 10-hour shift, and by the end of their shift the manufacturer still needs another 20 toys to be able to ship the order. How many toys per hour does the fifth worker produce?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
input_text="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: John drives for 3 hours at a speed of 60 mph and then turns around because he realizes he forgot something very important at home.  He tries to get home in 4 hours but spends the first 2 hours in standstill traffic.  He spends the next half-hour driving at a speed of 30mph, before being able to drive the remaining time of the 4 hours going at 80 mph.  How far is he from home at the end of those 4 hours?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Create attention mask
attention_mask = inputs.attention_mask

# 定义停止条件
stop_token_id = tokenizer.convert_tokens_to_ids(["</s>", "<|eot_id|>", "<|end_of_text|>", "<|end_header_id|>", "<|start_header_id|>"])
stopping_criteria = StoppingCriteriaList([StopTokenCriteria(stop_token_id)])


In [3]:
from hf_model_adapt import hf_adapt

input_length = inputs["input_ids"].shape[1]
### Todo: complete hf_adapt 
#(model, numDecodeLayer, numSkipLayer) = hf_adapt(model, tokenizer, 512, nBarLayer=54, valBarSim=0.98, nOutLayer = 4, nCheckLayer=2, nWarmupTok = input_length + 3, globalBarLayer=-1, verbose=True) 
#(model, numDecodeLayer, numSkipLayer) = hf_adapt(model, tokenizer, 512, nBarLayer=20, valBarSim=0.96, nOutLayer = 3, nCheckLayer=2, nWarmupTok = -1, globalBarLayer=-1, verbose=True)

(model, globalNumDecodedLayer, globalNumSkippedLayer) = bamboo(model, tokenizer, 1024, nBarLayer=24, valBarSim=0.96, nOutLayer = 3, nCheckLayer=2, nWarmupTok = -1, globalBarLayer=-1, verbose=True)

#import pdb; pdb.set_trace()
# Generate text
generation_kwargs = {
                        "do_sample":False,
                         "temperature":0, 
                         "top_p":1
                    }
outputs = model.generate(inputs["input_ids"], attention_mask=attention_mask, max_length=512, num_return_sequences=1, 
                         pad_token_id=tokenizer.eos_token_id, stopping_criteria=stopping_criteria, **generation_kwargs)

# Decode the generated text
generated_text = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=False)

print(f'\nPrompt::: {input_text}\n')
print(f'Response >>> {generated_text}')
print(f'\nNumDecodedLayer={torch.sum(globalNumDecodedLayer)}\tNumSkippedLayer={torch.sum(globalNumSkippedLayer)}')
print(f'SaveRatio={torch.sum(globalNumSkippedLayer)/(torch.sum(globalNumSkippedLayer)+torch.sum(globalNumDecodedLayer))*100:.2f}%')

/home/dozhang/miniconda3/envs/py31/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



*** Set BarLayer=25 based on prompt-layer similairty over 125 tokens.

--- No truncation at #position 124, #SimScore 0.953125, for token ['<|begin_of_text|>', '<|begin_of_text|>', '<|start_header_id|>', 'system', '<|end_header_id|>', 'ĊĊ', 'You', 'Ġare', 'Ġa', 'Ġhelpful', 'ĠAI', 'Ġassistant', '.', '<|eot_id|>', '<|start_header_id|>', 'user', '<|end_header_id|>', 'ĊĊ', 'Question', ':', 'ĠJohn', 'Ġdrives', 'Ġfor', 'Ġ', '3', 'Ġhours', 'Ġat', 'Ġa', 'Ġspeed', 'Ġof', 'Ġ', '60', 'Ġmph', 'Ġand', 'Ġthen', 'Ġturns', 'Ġaround', 'Ġbecause', 'Ġhe', 'Ġrealizes', 'Ġhe', 'Ġforgot', 'Ġsomething', 'Ġvery', 'Ġimportant', 'Ġat', 'Ġhome', '.', 'Ġ', 'ĠHe', 'Ġtries', 'Ġto', 'Ġget', 'Ġhome', 'Ġin', 'Ġ', '4', 'Ġhours', 'Ġbut', 'Ġspends', 'Ġthe', 'Ġfirst', 'Ġ', '2', 'Ġhours', 'Ġin', 'Ġstand', 'still', 'Ġtraffic', '.', 'Ġ', 'ĠHe', 'Ġspends', 'Ġthe', 'Ġnext', 'Ġhalf', '-hour', 'Ġdriving', 'Ġat', 'Ġa', 'Ġspeed', 'Ġof', 'Ġ', '30', 'mph', ',', 'Ġbefore', 'Ġbeing', 'Ġable', 'Ġto', 'Ġdrive', 'Ġthe', 'Ġremaining', 'Ġt